In [ ]:
import geopandas as gpd
from getpass import getpass
import numpy as np
from pathlib import Path
from pprint import pprint
import shutil
from typing import List, Union

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window

from rscube.nd_tools import (get_array_from_features, 
                             get_features_from_array, 
                             get_superpixel_area_as_features, 
                             get_superpixel_means_as_features,
                             get_superpixel_stds_as_features, 
                             scale_img)
from rscube.rio_tools import (get_geopandas_features_from_array, 
                              rasterize_shapes_to_array, 
                              get_indices_from_extent, get_cropped_profile)
from shapely.geometry import box
from tqdm.notebook import tqdm

from planet import api
from planet.api import downloader


import rioxarray as rxr
from shapely.geometry import mapping

This notebook downloads a Planet Scene and prepares it for the DSWx workflow.

# 1. Download Planet Data

In [ ]:
planet_api_key = getpass("Input your Planet API key")

In [ ]:
planet_scene_id = '20211003_161639_91_241d'
# planet_scene_id = input("Enter a Planet Scene ID")

In [ ]:
def planet_download(api_key: str, scene_id: str, output_dir: Union[str, Path] = Path.cwd(), asset_types: List = ['ortho_analytic_8b_sr']):
    """
    Download Planet data by scene ID 

    Keyword arguments:
    api_key     -- Planet API key
    scene_id    -- Planet data scene ID
    output_dir  -- path to directory in which to download data
    asset_types -- Planet data asset types to download
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    client = api.ClientV1(api_key=planet_api_key)
    request = client.get_item('PSScene', 
                          scene_id)
    items_to_download = [request.get()]

    downloader = api.downloader.create(client)
    downloader.activate(iter(items_to_download), asset_types)
    downloader.download(iter(items_to_download), asset_types, str(output_dir))
    print(downloader.stats())
    downloader.shutdown()

In [ ]:
planet_dir = Path.cwd()/"planet_data"

planet_download(planet_api_key, planet_scene_id, planet_dir)

# 2. Identify Intersecting Chip

**TODO: ACCOUNT FOR MULTIPLE CHIP INTERSECTIONS**

### Expose metadata

In [ ]:
planet_image_path = list(planet_dir.rglob(f'{planet_scene_id}*.tif'))[0]

with rasterio.open(planet_image_path) as ds:
    planet_crs = ds.crs
    planet_box = box(*ds.bounds)
    planet_bounds = list(ds.bounds)
    planet_profile = ds.profile
    planet_shape = planet_profile['height'], planet_profile['width']

print(f'Planet Imagery Shape: {planet_shape}')
print(f'Planet Imagery CRS: {planet_crs}')
print(f'Planet Image Profile: {planet_profile}')

### Read chips into a DataFrame and project to Planet data's UTM

In [ ]:
chips = gpd.read_file('6km_chips.geojson')
chips_utm = chips.to_crs(planet_crs)
chips_utm.head()

### Create DataFrame of chips intersecting with Planet Data

- currently assumes a single intersecting chip, which may not always be true

In [ ]:
intersects = chips_utm.geometry.intersects(planet_box) # create intersection mask
intersecting_chips = chips_utm[intersects].reset_index(drop=True) # create new Dataframe containing intersecting chips
chip_id = index = intersecting_chips.random_id[0]
intersecting_chips

# 3. Create Chip Shapefile  

In [ ]:
chips_dir = Path.cwd()/"chips"
chips_dir.mkdir(exist_ok=True)

chip_dir = chips_dir/f"chip_{chip_id}"
chip_dir.mkdir(exist_ok=True)

shape_path = chip_dir/f'chip_{index}.shp'

geom = chips.loc[chips['random_id'] == chip_id, 'geometry'].values[0]
df = gpd.GeoDataFrame({"id":1,"geometry":[geom]})
df.to_file(shape_path)

# 4. Preprocess Data


In [ ]:
crop_extent = chips.loc[chips['random_id'] == chip_id, 'geometry'].values[0]
crop_extent

In [ ]:
crop_extent_gdf = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[crop_extent])
crop_extent_gdf

### Clip Planet data to the chip extent 

In [ ]:
planet_image = rxr.open_rasterio(planet_image_path,
                                 masked=True).squeeze()

planet_clipped = planet_image.rio.clip(crop_extent_gdf.geometry.apply(mapping), 'EPSG:4326')

### Remove Coastal Blue no-data pixels from all Planet Imagery bands

In [ ]:
coastal_blue_clip = planet_clipped.where(planet_clipped[0] != planet_profile['nodata'])

### Subset Planet imagery to Red, Green, NIR bands

In [ ]:
rgnir = coastal_blue_clip.sel(band=[8, 6, 4])
rgnir.attrs['long_name'] = ('nir', 'red', 'green')
rgnir

### Remove outliers (highest and lowest 2% pixel values) from all 3 bands and normalize

In [ ]:
rgnir_out = rgnir.copy()
for i in tqdm(range(3)):
    m0 = rgnir_out[i].quantile(0.02)
    m1 = rgnir_out[i].quantile(0.98)
    
    rgnir_out[i] = rgnir_out[i].clip(m0, m1)
    
    rgnir_out[i] = scale_img(rgnir_out[i])

# 5. Write Data to GeoTiff

In [ ]:
tif_path = chip_dir/f'Planet_nir_red_green_chip_{chip_id}.tif'

rgnir_out.rio.to_raster(tif_path)